In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np

In [2]:
LOAD_MODEL = True

In [3]:
dataset = pd.read_csv('processed_data.csv').iloc[:,:2]
dataset.columns.values[0]='spam'

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\boroi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\boroi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
nltk.tokenize.word_tokenize('hello my friend')

['hello', 'my', 'friend']

In [7]:
EXPECTED = []
filtered_strings  = []
for index, row in dataset.iterrows():
    try:
        word_tokens = word_tokenize(row[1])
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        filtered_strings.append(' '.join(filtered_sentence))
        EXPECTED.append(row[0])
    except:
        pass

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

if not LOAD_MODEL:
    tokenizer = Tokenizer(num_words = 10000,oov_token='<OOV>')
    tokenizer.fit_on_texts(filtered_strings)
    
    tokenizer_json = tokenizer.to_json()
    with open('tokenizer.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))
        
    print('Tokenizer Created')    
else:
    with open('tokenizer.json') as f:
        data = json.load(f)
        tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)
        
    print('Tokenizer Loaded')    
    

Tokenizer Loaded


In [9]:
SET_DATA = pad_sequences(tokenizer.texts_to_sequences(filtered_strings),maxlen=200)

In [10]:
from tensorflow.keras.models import Sequential

In [11]:
from keras.optimizers import SGD

if not LOAD_MODEL:
    model = Sequential([
        tf.keras.layers.Dense(70,input_shape=(200,),activation='relu'),
        tf.keras.layers.Dense(30,activation='relu'),
        tf.keras.layers.Dense(10,activation='relu'),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    model.compile(loss='mean_absolute_error',optimizer=SGD(lr=0.01),metrics=['accuracy'])
else:
    model = tf.keras.models.load_model('model.h5')    
    print('Model loaded')
model.summary()

Model loaded
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 70)                14070     
                                                                 
 dense_1 (Dense)             (None, 30)                2130      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 16,521
Trainable params: 16,521
Non-trainable params: 0
_________________________________________________________________


In [12]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if(epoch % 200 == 0):
            print("Saving model")
            model.save('model.h5')

In [13]:
print(len(SET_DATA),len(EXPECTED))

74626 74626


In [18]:
model.fit(np.array(SET_DATA),np.array(EXPECTED),batch_size=34626,epochs=5000,callbacks=[CustomCallback()])

Epoch 1/5000
3/3 [==============================] - 0s 41ms/step - loss: 0.2513 - accuracy: 0.7490
Epoch 2/5000
3/3 [==============================] - 0s 19ms/step - loss: 0.2633 - accuracy: 0.7369
Epoch 3/5000
3/3 [==============================] - 0s 19ms/step - loss: 0.2545 - accuracy: 0.7457
Epoch 4/5000
3/3 [==============================] - 0s 20ms/step - loss: 0.2530 - accuracy: 0.7473
Epoch 5/5000
3/3 [==============================] - 0s 18ms/step - loss: 0.2530 - accuracy: 0.7474
Epoch 6/5000
3/3 [==============================] - 0s 18ms/step - loss: 0.2545 - accuracy: 0.7455
Epoch 7/5000
3/3 [==============================] - 0s 18ms/step - loss: 0.2530 - accuracy: 0.7473
Epoch 8/5000
3/3 [==============================] - 0s 19ms/step - loss: 0.2594 - accuracy: 0.7408
Epoch 9/5000
3/3 [==============================] - 0s 18ms/step - loss: 0.2546 - accuracy: 0.7455
Epoch 10/5000
3/3 [==============================] - 0s 19ms/step - loss: 0.2522 - accuracy: 0.7481
Epoch 11/